In [ ]:
!pip install geopandas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from datetime import datetime

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
map = {
    'severe': 3,
    'moderate': 2,
    'mild': 1
    }

def getWeekVector(fin_df,wkCt,year):
  # print("after ", new_df.head(10))
  for index, row in fin_df.iterrows():
    ds = row[f'dryspell_{year}_week_{wkCt}']
    rfdev = row[f'monthly_rainfall_deviation_{year}_week_{wkCt}']
    spi = row[f'spi_{year}_week_{wkCt}']
    vci = row[f'vci_{year}_week_{wkCt}']
    mai = row[f'mai_{year}_week_{wkCt}']
    cas = row[f'kharif_cropped_sqkm_{year}']
    # print(row,"==>",ds,rfdev,spi,vci,mai,cas)

    vci_class = 'mild'
    if(60<vci and vci<=100):
      vci_class='severe'
    elif(40<vci and vci<=60):
      vci_class='moderate'
    else:
      vci_class='mild'

    mai_class = 'mild'
    if(mai<=25):
      mai_class = 'severe'
    elif(mai<=50):
      mai_class = 'moderate'
    else:
      mai_class = 'mild'

    cas_class = 'mild'
    if(cas<=33.3):
      cas_class = 'severe'
    elif(cas<=50):
      cas_class = 'moderate'
    else:
      cas_class = 'mild'

    mD = 0
    if(ds==1 or rfdev=='scanty' or spi<-1.5):
      mD = 1
    if(mD==1):
      if(vci_class=='severe' and mai_class=='severe' and cas_class=='severe'):
        if(ds==1):
          fin_df.at[index,f'severe_drought_path1_{year}']+=1
        elif(rfdev=='scanty'):
          fin_df.at[index,f'severe_drought_path2_{year}']+=1
        else:
          fin_df.at[index,f'severe_drought_path3_{year}']+=1
      elif(vci_class=='mild' and mai_class=='severe' and cas_class=='severe'):
        if(ds==1):
          fin_df.at[index,f'moderate_drought_path1_{year}']+=1
        elif(rfdev=='scanty'):
          fin_df.at[index,f'moderate_drought_path2_{year}']+=1
        else:
          fin_df.at[index,f'moderate_drought_path3_{year}']+=1
      elif(vci_class=='moderate' and mai_class=='severe' and cas_class=='severe'):
        if(ds==1):
          fin_df.at[index,f'moderate_drought_path4_{year}']+=1
        elif(rfdev=='scanty'):
          fin_df.at[index,f'moderate_drought_path5_{year}']+=1
        else:
          fin_df.at[index,f'moderate_drought_path6_{year}']+=1
      elif(vci_class=='severe' and mai_class=='mild' and cas_class=='severe'):
        if(ds==1):
          fin_df.at[index,f'moderate_drought_path7_{year}']+=1
        elif(rfdev=='scanty'):
          fin_df.at[index,f'moderate_drought_path8_{year}']+=1
        else:
          fin_df.at[index,f'moderate_drought_path9_{year}']+=1
      elif(vci_class=='severe' and mai_class=='moderate' and cas_class=='severe'):
        if(ds==1):
          fin_df.at[index,f'moderate_drought_path10_{year}']+=1
        elif(rfdev=='scanty'):
          fin_df.at[index,f'moderate_drought_path11_{year}']+=1
        else:
          fin_df.at[index,f'moderate_drought_path12_{year}']+=1
      elif(vci_class=='severe' and mai_class=='severe' and cas_class=='mild'):
        if(ds==1):
          fin_df.at[index,f'moderate_drought_path13_{year}']+=1
        elif(rfdev=='scanty'):
          fin_df.at[index,f'moderate_drought_path14_{year}']+=1
        else:
          fin_df.at[index,f'moderate_drought_path15_{year}']+=1
      elif(vci_class=='severe' and mai_class=='severe' and cas_class=='moderate'):
        if(ds==1):
          fin_df.at[index,f'moderate_drought_path16_{year}']+=1
        elif(rfdev=='scanty'):
          fin_df.at[index,f'moderate_drought_path17_{year}']+=1
        else:
          fin_df.at[index,f'moderate_drought_path18_{year}']+=1
      else:
        if(ds==1):
          fin_df.at[index,f'mild_drought_dryspell_score_{year}']+=1
        elif(rfdev=='scanty'):
          fin_df.at[index,f'mild_drought_rainfall_deviation_score_{year}']+=1
        elif(spi<-1.5):
          fin_df.at[index,f'mild_drought_spi_score_{year}']+=1
        fin_df.at[index,f'mild_drought_vci_score_{year}']+=map[vci_class]
        fin_df.at[index,f'mild_drought_mai_score_{year}']+=map[mai_class]
        fin_df.at[index,f'mild_drought_cropping_area_sown_score_{year}']+=map[cas_class]
"""
VCI Value (%) Vegetation Condition
60-100 Good
40-60 Fair
0-40 Poor


MAI (%) Agricultural Drought Class
51-100 Mild drought
26-50 Moderate drought
0-25 Severe drought

Area Sown(%) Drought Condition
0-33.3 Severe drought
33.3-50 Moderate drought
50-100 Mild drought

"""


'\nVCI Value (%) Vegetation Condition\n60-100 Good\n40-60 Fair\n0-40 Poor\n\n\nMAI (%) Agricultural Drought Class\n51-100 Mild drought\n26-50 Moderate drought\n0-25 Severe drought\n\nArea Sown(%) Drought Condition\n0-33.3 Severe drought\n33.3-50 Moderate drought\n50-100 Mild drought\n\n'

In [ ]:
block_name = 'Mohanpur'
shp_path =  f'/content/drive/MyDrive/drought_modisv2/{block_name}.shp'
gdf = gpd.read_file(shp_path)
# print(gdf.head(10))
filepath = f'/content/drive/MyDrive/drought_modis_mask/{block_name}_allvar.csv'
df = pd.read_csv(filepath)
# dd = [col for col in df.columns if '2003' in col]
# for col in dd:
#   print(col)

# Dates = []

labels = ['dryspell','mai','meteorological_drought', 'monthly_rainfall_deviation', 'spi', 'rfdev_class', 'vci']
singleLabels = ['monsoon_onset', 'total_weeks', 'kharif_cropped_sqkm']

create_cols = ['frequency_of_no_drought', 'frequency_of_mild_drought', 'frequency_of_moderate_drought', 'frequency_of_severe_drought',
               'intensity_of_no_drought', 'intensity_of_mild_drought', 'intensity_of_moderate_drought', 'intensity_of_severe_drought',
               'severe_drought_path1', 'severe_drought_path2', 'severe_drought_path3',
               'moderate_drought_path1', 'moderate_drought_path2', 'moderate_drought_path3', 'moderate_drought_path4', 'moderate_drought_path5',
               'moderate_drought_path6', 'moderate_drought_path7', 'moderate_drought_path8', 'moderate_drought_path9', 'moderate_drought_path10',
               'moderate_drought_path11', 'moderate_drought_path12', 'moderate_drought_path13', 'moderate_drought_path14', 'moderate_drought_path15',
               'moderate_drought_path16', 'moderate_drought_path17', 'moderate_drought_path18',
               'mild_drought_dryspell_score', 'mild_drought_rainfall_deviation_score', 'mild_drought_spi_score',
               'mild_drought_vci_score', 'mild_drought_mai_score', 'mild_drought_cropping_area_sown_score'
               ]

fin_df = pd.DataFrame()
fin_df['uid'] = df['uid']

for year in range(2003,2023):
  for col in create_cols:
    fin_df[f'{col}_{year}'] = 0
  fin_df[f'frequency_of_no_drought_{year}'] = df[f'freq_of_drought_{year}_at_threshold_0']
  fin_df[f'frequency_of_mild_drought_{year}'] = df[f'freq_of_drought_{year}_at_threshold_1']
  fin_df[f'frequency_of_moderate_drought_{year}'] = df[f'freq_of_drought_{year}_at_threshold_2']
  fin_df[f'frequency_of_severe_drought_{year}'] = df[f'freq_of_drought_{year}_at_threshold_3']
  fin_df[f'intensity_of_no_drought_{year}'] = df[f'intensity_of_drought_{year}_at_threshold_0']
  fin_df[f'intensity_of_mild_drought_{year}'] = df[f'intensity_of_drought_{year}_at_threshold_1']
  fin_df[f'intensity_of_moderate_drought_{year}'] = df[f'intensity_of_drought_{year}_at_threshold_2']
  fin_df[f'intensity_of_severe_drought_{year}'] = df[f'intensity_of_drought_{year}_at_threshold_3']

  prefixName = f"weekly_label_{year}"
  # print(prefixName)
  dates = [col[13:] for col in df.columns if (prefixName in col)]
  # print(dates)
  date_objects = [[datetime.strptime(date, '%Y-%m-%d'),date] for date in dates]
  # Sort datetime objects
  sorted_dates = sorted(date_objects)
  sorted_dates_strings = [date[1] for date in sorted_dates]
  # print(len(sorted_dates_strings))
  # print( df[f'total_weeks_{year}'])

  # new_df = pd.DataFrame()
  # new_df['uid'] = df['uid']

  new_cols = []
  for label in singleLabels:
    colname = f'{label}_{year}'
    fin_df[colname]=df[colname]
    new_cols.append(colname)

  for label in labels:
    for i in range(len(sorted_dates_strings)):
      s=sorted_dates_strings[i]
      col = f'{label}_{s}'
      colnew = f'{label}_{year}_week_{i+1}'
      fin_df[colnew] = df[col]
      new_cols.append(colnew)
  # for col in new_df.columns:
    # print(col)

  for i in range(len(sorted_dates_strings)):
    weekCt = i+1
    # new_df[f'vector_{year}_week_{weekCt}'] = np.nan
    # print("before ", new_df.head(10))
    getWeekVector(fin_df,weekCt,year)
    # print(new_df[f'vector_{year}_week_{weekCt}'])

  fin_df.drop(new_cols,axis=1,inplace=True)
  fin_df[f'mild_drought_vci_score_{year}']/=6
  fin_df[f'mild_drought_mai_score_{year}']/=6
  fin_df[f'mild_drought_cropping_area_sown_score_{year}']/=6


<ipython-input-8-3ff660a6dcba>:67: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fin_df[colnew] = df[col]
<ipython-input-8-3ff660a6dcba>:67: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fin_df[colnew] = df[col]
<ipython-input-8-3ff660a6dcba>:67: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fin_df[c

In [ ]:
print(fin_df.columns.tolist())

['uid', 'frequency_of_no_drought_2003', 'frequency_of_mild_drought_2003', 'frequency_of_moderate_drought_2003', 'frequency_of_severe_drought_2003', 'intensity_of_no_drought_2003', 'intensity_of_mild_drought_2003', 'intensity_of_moderate_drought_2003', 'intensity_of_severe_drought_2003', 'severe_drought_path1_2003', 'severe_drought_path2_2003', 'severe_drought_path3_2003', 'moderate_drought_path1_2003', 'moderate_drought_path2_2003', 'moderate_drought_path3_2003', 'moderate_drought_path4_2003', 'moderate_drought_path5_2003', 'moderate_drought_path6_2003', 'moderate_drought_path7_2003', 'moderate_drought_path8_2003', 'moderate_drought_path9_2003', 'moderate_drought_path10_2003', 'moderate_drought_path11_2003', 'moderate_drought_path12_2003', 'moderate_drought_path13_2003', 'moderate_drought_path14_2003', 'moderate_drought_path15_2003', 'moderate_drought_path16_2003', 'moderate_drought_path17_2003', 'moderate_drought_path18_2003', 'mild_drought_dryspell_score_2003', 'mild_drought_rainfall

In [ ]:
# Just for Creating CSV
fin_df.to_csv(f'/content/drive/MyDrive/drought_modis_mask/{block_name}_kyl.csv', index=False)